In [1]:
import pandas as pd
import numpy as np
import pycmap
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import os
import glob
import plotly.express as px

In [3]:
## also skip 13 and 14
mat_code = pd.read_csv('lookup_table_fixed.txt', sep = '  ')
mat_code

/var/folders/rl/0vwydsgj6nx5gmddmc5qtxz00000gn/T/ipykernel_572/3311222237.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  mat_code = pd.read_csv('lookup_table_fixed.txt', sep = '  ')


,thetasun,Unnamed: 1,Ed(air),Unnamed: 3,Eu(air) PAR_Ed(air) PAR_Eo(air),Unnamed: 5,Ed(z=0),Unnamed: 7,Eu(z=0) PAR_Ed(z=0) PAR_Eo(z=0)
0,0.0,1.00000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,116.94000,7.040100,535.3500,1048.3000,112.00000,2.095500,512.0200,626.8500
2,2.0,116.86000,7.043600,534.9800,1048.6000,111.91000,2.094300,511.6300,626.4600
3,4.0,116.62000,7.038400,533.8600,1047.6000,111.67000,2.090100,510.5100,625.2100
4,6.0,116.21000,7.024700,531.9900,1045.3000,111.26000,2.083000,508.6800,623.1100
...,...,...,...,...,...,...,...,...,...
230,82.0,8.69280,0.499680,39.7190,83.0700,8.36410,0.170970,38.1570,49.4250
231,84.0,5.89570,0.341090,26.7210,55.9440,5.67580,0.121240,25.6830,33.3480
232,86.0,3.58860,0.209480,16.0680,33.6720,3.45760,0.078444,15.4560,20.1410
233,88.0,1.69650,0.099837,7.5046,15.7340,1.63580,0.039141,7.2245,9.4488


%% Extra processing - KD490 and derived KPar

% KdPAR = f(kd490) = f(chl)

ttG5.kd490 = 0.0166 + 0.0773.*ttG5.lhchl.^0.6715;

ttG5.kdPar = 0.0665 + 0.874.*ttG5.kd490 - 0.00121./ttG5.kd490;

%Air-sea transmission of scalar par

%This is a function of sun zenith angle and wind speed (Mobley and Boss 2012)

load 'sza_clear_chl0d5.mat'

Fscalar = scatteredInterpolant(sza_clear(:,1),sza_clear(:,2),sza_clear(:,10)./sza_clear(:,5));

transAirWater = Fscalar(ttG5.solZen,ttG5.Wind./1.94384);

% Calculate Par at 5m depth
ttG5.par5m = transAirWater.*ttG5.Par.*exp(-ttG5.kdPar.*5); 

% 5m for the Thompson

In [ ]:
## % signs are comments in matlab; %% signs are new section in matlab code
## is kd490 column name for 490 nm backscatter ?
## is ttG5 thompson gradients 5 and is it the dataframe ?
## not sure what kd means
## idk what the dot means hehe

## Extra processing - KD490 and derived KPar

In [ ]:
# KdPAR = f(kd490) = f(chl) ## KdPAR (the attenuation factor at a certain depth for PAR?) is equivalent to function of chloro
# .* in matlab is to multiply elements of the same length togehter
# .* power function of elements with same length
# ./ is right side division (left side of element gets divded by right side of element)

# scatteredinterpolant creates empty scattered data object (then filled with specific data)

ttg5['kd490'] =  0.0166 + 0.0773 * ttg5['lhchl']^0.6715
ttg5['kdPar'] = 0.0665 + 0.874 * ttg5['kd490'] - (0.00121 / ttg5['kd490'])

mat_code = pd.read_csv('lookup_table_fixed.txt', sep = '  ')

Fscalar = 